In [1]:
import time            
import re            
import os    
import sys  
import codecs  
import shutil  
import nltk
import numpy as np  
import matplotlib  
import scipy  
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn import feature_extraction    
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
#title
asins = open('../CategoryCluster/C10/In/pre/ASIN.csv').read().split('\n')
ASINs=[]
for item in asins:
    a = item.split(',')
    ASINs += a
# ASINs = ASINs[:-1]
len(ASINs)

7100

In [3]:
#synopses
review = open('../CategoryCluster/C10/In/pre/review.csv').read().split('\n')
review[0]
Reviews=[]
for rev in review:
    a = rev.split('|')
    Reviews += a 
Reviews = Reviews[:-1]
len(Reviews)


7100

In [4]:
#ranks
ranks = open('../CategoryCluster/C10/In/pre/rank.csv').read().split('\n')
ranks[0] = ranks[0].replace('\ufeff','')
Rank =[]
for rank in ranks:
    a = rank.split(',')
    Rank += a
    
Ranks =[float(i) for i in Rank]  
Ranks[0]
len(Ranks)

7100

In [5]:
# 载入 nltk 的英文停用词作为“stopwords”变量
stopwords = nltk.corpus.stopwords.words('english')

In [6]:
# 载入 nltk 的 SnowballStemmer 作为“stemmer”变量
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [7]:
# 这里我定义了一个分词器（tokenizer）和词干分析器（stemmer），它们会输出给定文本词干化后的词集合
 
def tokenize_and_stem(text):
    # 首先分句，接着分词，而标点也会作为词例存在
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # 过滤所有不含字母的词例（例如：数字、纯标点）
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems
 
def tokenize_only(text):
    # 首先分句，接着分词，而标点也会作为词例存在
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # 过滤所有不含字母的词例（例如：数字、纯标点）
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [8]:
# 非常不 pythonic，一点也不！
# 扩充列表后变成了非常庞大的二维（flat）词汇表
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in Reviews:
    allwords_stemmed = tokenize_and_stem(i) #对每个评价进行分词和词干化
    totalvocab_stemmed.extend(allwords_stemmed) # 扩充“totalvocab_stemmed”列表
 
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

## Tf-idf and document similarity

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
# 定义向量化参数
#将文本中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在第i类评论下的词频  
vectorizer = CountVectorizer(max_df=0.98, max_features=200000,
                                 min_df=0.02, stop_words='english',
                                 tokenizer=tokenize_and_stem, ngram_range=(1,3))
#该类会统计每个词语的tf-idf权值  
transformer = TfidfTransformer() 

#第一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵  
%time tf =  vectorizer.fit_transform(Reviews)
%time tfidf = transformer.fit_transform(tf) # 向量化
 

CPU times: user 3.6 s, sys: 34.9 ms, total: 3.64 s
Wall time: 3.69 s
CPU times: user 2.32 ms, sys: 463 µs, total: 2.78 ms
Wall time: 2.5 ms


In [10]:
#获取词袋模型中的前10个词语 
word = vectorizer.get_feature_names()
print(word[:10])

["'s", 'amazon', 'awesom', 'babi', 'bad', 'bit', 'buy', 'color', 'colour', 'cute']


In [11]:
# 词袋模型中共有几个词语
len(word)

56

In [12]:
#将tf-idf矩阵抽取出来，元素w[i][j]表示j词在第i类评论中的tf-idf权重  
weight = tfidf.toarray()  
weight

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.20914305,  0.        ],
       [ 0.41527425,  0.        ,  0.49553066, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.8747019 , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [13]:
weight.shape

(7100, 56)

In [14]:
#词频矩阵
tfidframe = pd.DataFrame(np.round(weight, 2))

In [15]:
tfidframe[:10]

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,0.00,0.0,0.00,0.0,0.0,0.23,0.2,0.22,0.0,0.0,...,0.0,0.43,0.00,0.0,0.0,0.24,0.13,0.0,0.21,0.0
1,0.42,0.0,0.50,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0
2,0.00,0.0,0.87,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0
3,0.00,0.3,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.33,0.65,0.0,0.0,0.00,0.00,0.0,0.00,0.0
4,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.41,0.0,0.00,0.0
5,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0
6,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0
7,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0
8,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0
9,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0


## cosine similarity

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(tfidf)
dist = 1 - similarity

In [17]:
print(dist)

[[ 0.          1.          1.         ...,  1.          1.          1.        ]
 [ 1.          0.          0.43348624 ...,  1.          1.          1.        ]
 [ 1.          0.43348624  0.         ...,  1.          1.          1.        ]
 ..., 
 [ 1.          1.          1.         ...,  1.          1.          1.        ]
 [ 1.          1.          1.         ...,  1.          1.          1.        ]
 [ 1.          1.          1.         ...,  1.          1.          1.        ]]


In [18]:
similarity_df = pd.DataFrame(similarity)
similarity_df[:10]

,0,1,2,3,4,5,6,7,8,9,...,7090,7091,7092,7093,7094,7095,7096,7097,7098,7099
0,1.000000,0.000000,0.000000,0.296496,0.143971,0.31469,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,1.000000,0.566514,0.000000,0.102739,0.00000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.566514,1.000000,0.000000,0.181353,0.00000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.296496,0.000000,0.000000,1.000000,0.000000,0.49780,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.143971,0.102739,0.181353,0.000000,1.000000,0.00000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.314690,0.000000,0.000000,0.497800,0.000000,1.00000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,1.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,1.0,...,0.600609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## K-means clustering

In [19]:
from sklearn.cluster import KMeans  
num_clusters = 10
km = KMeans(n_clusters=10)   
%time s = km.fit(weight)  
clusters = km.labels_.tolist()
print(s)

CPU times: user 416 ms, sys: 25.7 ms, total: 442 ms
Wall time: 397 ms
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [20]:
from sklearn.externals import joblib
# 注释语句用来存储你的模型
# 因为我已经从 pickle 载入过模型了
 
joblib.dump(km,  'doc_cluster.pkl')
 
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [21]:
toys = { 'title': ASINs, 'rank': Ranks, 'review': Reviews, 'cluster': clusters}
 
frame = pd.DataFrame(toys , index = [clusters] , columns = ['rank', 'title', 'cluster'])

In [22]:
# frame[:5]

In [23]:
# frame['cluster'].value_counts() #number of review per cluster (clusters from 0 to 10)

In [24]:
# km.inertia_

In [25]:
# grouped = frame['rank'].groupby(frame['cluster']) # 为了凝聚（aggregation），由聚类分类。
 
# grouped.mean() # 每个聚类的平均排名（0 到 5）

In [26]:
# from __future__ import print_function
 
# print("Top terms per cluster:")
# print()
# # 按离质心的距离排列聚类中心，由近到远
# order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
 
# for i in range(num_clusters):
#     print("Cluster %d titles:" % i, end='')
#     for title in frame.ix[i]['title'].values.tolist():
#         print(' %s,' % title, end='')
#     print() # 空行
#     print() # 空行

In [27]:
# from __future__ import print_function
 
# print("Top terms per cluster:")
# print()
# # 按离质心的距离排列聚类中心，由近到远
# order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
# # print(order_centroids)
# for i in range(num_clusters):
#     print("Cluster %d:" % i, end='')
#     time.sleep(1)
#     for ind in order_centroids[i, :50]: # 每个聚类选 10 个词
#         print(' %s' % word[ind]  , end=',')
#     print() # 空行
#     print() # 空行

## LDA

In [28]:
from sklearn.decomposition import LatentDirichletAllocation

In [29]:
n_topics = 10
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [30]:
%time lda.fit(tf) #tf 为向量化后的语料词集

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 33.2 s, sys: 168 ms, total: 33.4 s
Wall time: 33.7 s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=10, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [31]:
%time lda_matrix = lda.fit_transform(tf) #tf 为向量化后的语料词集

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 34.2 s, sys: 210 ms, total: 34.5 s
Wall time: 34.8 s


In [32]:
lda_matrix #表示每一個文檔屬於每一個聚類的概率

array([[ 0.00500022,  0.00500001,  0.00500153, ...,  0.9549917 ,
         0.00500105,  0.00500247],
       [ 0.84998967,  0.01666667,  0.01666667, ...,  0.01666667,
         0.0166678 ,  0.01666726],
       [ 0.69997989,  0.03333333,  0.03333333, ...,  0.03333333,
         0.03333828,  0.03333672],
       ..., 
       [ 0.1       ,  0.1       ,  0.1       , ...,  0.1       ,
         0.1       ,  0.1       ],
       [ 0.1       ,  0.1       ,  0.1       , ...,  0.1       ,
         0.1       ,  0.1       ],
       [ 0.1       ,  0.1       ,  0.1       , ...,  0.1       ,
         0.1       ,  0.1       ]])

In [33]:
features = pd.DataFrame(lda_matrix, columns=['T1', 'T2', 'T3','T4','T5','T6','T7','T8','T9','T10'])

In [34]:
features[:5]

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10
0,0.005000,0.005000,0.005002,0.005001,0.005001,0.005001,0.005000,0.954992,0.005001,0.005002
1,0.849990,0.016667,0.016667,0.016667,0.016671,0.016669,0.016668,0.016667,0.016668,0.016667
2,0.699980,0.033333,0.033333,0.033333,0.033333,0.033335,0.033344,0.033333,0.033338,0.033337
3,0.011111,0.011111,0.011111,0.566666,0.011111,0.011111,0.011111,0.011112,0.344443,0.011111
4,0.020007,0.020000,0.020000,0.020004,0.020001,0.819964,0.020013,0.020000,0.020006,0.020004


In [35]:
%time km_lda =km.fit_transform(features)

CPU times: user 185 ms, sys: 14.1 ms, total: 199 ms
Wall time: 189 ms


In [36]:
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])

In [37]:
ldaframe = pd.DataFrame({ 'title': ASINs, 'rank': Ranks, 'review': Reviews})
ldaframe = ldaframe[['title','rank','review']]
ldaClusterframe = pd.concat([ldaframe, cluster_labels], axis=1)
ldaClusterframe[:5]

,title,rank,review,ClusterLabel
0,B075BKMW23_1,5.0,""" Very good quality teddy, soft and size is 3 ...",6
1,B075BKMW23_2,5.0,"It's awesome product, great value for the mone...",0
2,B075BKMW23_3,4.0,mention proper description in product or else ...,0
3,B075BKMW23_4,5.0,Awsome. I got the same teddy bear as shown in ...,1
4,B075BKMW23_5,4.0,The product was just as described. Very soft n...,8


In [38]:
ldaClusterframe['ClusterLabel'].value_counts()

1    1987
4    1200
3     670
7     558
2     534
8     499
6     416
9     415
5     411
0     410
Name: ClusterLabel, dtype: int64

In [39]:
Rankgrouped = ldaClusterframe['rank'].groupby(ldaClusterframe['ClusterLabel']) # 为了凝聚（aggregation），由聚类分类。
 
Rankgrouped.mean() # 每个聚类的平均排名（0 到 5）

ClusterLabel
0    4.134146
1    1.400101
2    3.239700
3    4.291045
4    3.611667
5    4.783455
6    2.884615
7    3.354839
8    4.214429
9    3.291566
Name: rank, dtype: float64

In [40]:
ldaClusterframe['rank'].describe()

count    7100.000000
mean        3.087746
std         1.933838
min         0.000000
25%         1.000000
50%         4.000000
75%         5.000000
max         5.000000
Name: rank, dtype: float64

In [41]:
#排序
ldaClusterframe.sort_values(by = 'ClusterLabel')[:10]

,title,rank,review,ClusterLabel
4112,B0753JWCZD_3,4.0,mention proper description in product or else ...,0
5112,B01N00ZRE4_3,5.0,It's really good,0
4701,B072DZ63LW_2,5.0,Too good....larger than expected,0
1645,B075P6RNTL_6,5.0,Wow awsm product I really like it 😁 thanks to ...,0
5125,B01GCPZOZM_6,4.0,Good delivered as expected. Children happy.,0
4670,B01N0Z2TDI_1,5.0,Great product and totally worth the price. Goo...,0
4666,B072LCWMFR_7,4.0,Its good as i had expected,0
2557,B0716S97FM_8,4.0,Its good as i had expected,0
6250,B072LCWMFR_1,5.0,Gud job .!Delivery was 2 day before scheduled ...,0
6252,B072LCWMFR_3,4.0,Reached the expectations,0


In [42]:
from __future__ import print_function
 
order_centroids =  lda.components_.argsort()[:, ::-1] 
# print(order_centroids)

for i in range(num_clusters):
#     print("Cluster %d:" % i, end='\n')
    time.sleep(1)
    for ind in order_centroids[i, :30]: # 每个聚类选 30 个词
        print(' %s' % word[ind], end=',')
        
        
    print() # 空行
    print() # 空行

 product, expect, realli, awesom, great, order, stitch, 's, good, deliv, price, money, got, happi, qualiti, like, veri, amazon, worth, bit, shown, differ, just, cute, look, good product, babi, bad, littl, size,

 love, gift, excel, littl, color, product, babi, veri, kid, buy, realli, qualiti, just, good, awesom, teddi, soft, price, worth, cute, veri good, thank, got, toy, amazon, great, time, size, like, look,

 small, kid, size, good, happi, babi, littl, use, bit, soft, veri, price, time, good product, product, like, look, 's, toy, just, qualiti, gift, worth, stitch, receiv, materi, thank, money, realli, amazon,

 amazon, thank, receiv, got, teddi, just, shown, excel, product, veri, deliv, shown pictur, time, pictur, 's, order, love, awesom, happi, good, look, like, bad, qualiti, realli, good qualiti, size, cute, great, gift,

 toy, like, worth, colour, veri, money, soft, look, babi, buy, price, got, bad, happi, good, 's, amazon, qualiti, product, kid, just, order, doe, deliv, stitch,

In [43]:
order_centroids

array([[36, 14, 39,  2, 20, 33, 46,  0, 16, 10, 35, 29, 19, 21, 37, 24, 52,
         1, 55,  5, 41, 11, 22,  9, 26, 17,  3,  4, 25, 43, 34,  7, 30, 15,
        40, 44, 28,  8, 47,  6, 48, 38, 50, 45, 12, 49, 51, 18, 23, 31, 27,
        13, 42, 54, 32, 53],
       [27, 15, 13, 25,  7, 36,  3, 52, 23,  6, 39, 37, 22, 16,  2, 47, 45,
        35, 55,  9, 54, 48, 19, 50,  1, 20, 49, 43, 24, 26, 28, 41, 21,  5,
         0, 40, 31, 53, 46, 17, 12, 44, 18, 51, 34, 30, 38, 29, 32, 14, 33,
        10,  4, 42,  8, 11],
       [44, 23, 43, 16, 21,  3, 25, 51,  5, 45, 52, 35, 49, 17, 36, 24, 26,
         0, 50, 22, 37, 15, 55, 46, 40, 28, 48, 29, 39,  1, 14,  9, 31, 12,
        19, 30, 27, 38,  4, 47, 41, 18, 33, 20,  6,  8,  7,  2, 32, 13, 34,
        10, 54, 53, 11, 42],
       [ 1, 48, 40, 19, 47, 22, 41, 13, 36, 52, 10, 42, 49, 34,  0, 33, 27,
         2, 21, 16, 26, 24,  4, 37, 39, 18, 43,  9, 20, 15,  7, 30, 35, 38,
        25, 29, 12,  6,  3, 44, 28,  8, 50, 23, 45, 55, 46, 11, 17, 14, 31,
 

In [44]:
ftf = transformer.fit_transform(order_centroids)
ftf = ftf.toarray()

In [45]:
#词频矩阵
tfidMMframe = pd.DataFrame(np.round(ftf, 2))
tfidMMframe[:5]

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,0.15,0.06,0.16,0.01,0.08,0.14,0.19,0.00,0.07,0.05,...,0.21,0.07,0.10,0.13,0.11,0.05,0.17,0.22,0.13,0.22
1,0.11,0.06,0.05,0.11,0.03,0.15,0.01,0.23,0.09,0.03,...,0.16,0.12,0.13,0.06,0.13,0.04,0.02,0.17,0.03,0.04
2,0.18,0.09,0.18,0.07,0.09,0.01,0.10,0.23,0.02,0.20,...,0.03,0.01,0.13,0.05,0.14,0.04,0.22,0.22,0.05,0.17
3,0.00,0.20,0.17,0.09,0.19,0.09,0.17,0.06,0.15,0.24,...,0.19,0.05,0.07,0.06,0.13,0.22,0.21,0.02,0.22,0.13
4,0.21,0.10,0.23,0.04,0.21,0.12,0.19,0.12,0.01,0.03,...,0.22,0.14,0.07,0.05,0.17,0.07,0.19,0.13,0.01,0.16


In [46]:
from __future__ import print_function
 
order_centroids =  lda.components_.argsort()[:, ::-1] 
# print(order_centroids)

for i in range(num_clusters):
    print("Cluster %d:" % i, end='\n')
    time.sleep(1)
    for ind in order_centroids[i, :30]: # 每个聚类选 30 个词
        print(' %s' % word[ind], end=',')
        print(' %f' % lda.components_[i][ind-1], end=',')
        print(' %f' % (tfidMMframe[i:i+1][ind]), end=',')#a词在所属群中出现的频率
        print(' %f' % max(tfidframe[:][ind]), end=',') #a词与所有顾客评论中出现最高词频的相对指数
        print(' %f' % (tfidMMframe[i:i+1][ind]/max(tfidframe[:][ind])), end='\n')
        
    print('') # 空行
    print() # 空行

Cluster 0:
 product, 48.302188, 0.120000, 1.000000, 0.120000
 expect, 0.100002, 0.170000, 1.000000, 0.170000
 realli, 0.100006, 0.020000, 1.000000, 0.020000
 awesom, 0.100012, 0.160000, 1.000000, 0.160000
 great, 0.100020, 0.170000, 1.000000, 0.170000
 order, 0.100001, 0.060000, 0.860000, 0.069767
 stitch, 0.100006, 0.210000, 1.000000, 0.210000
 's, 0.100012, 0.150000, 1.000000, 0.150000
 good, 0.100008, 0.220000, 1.000000, 0.220000
 deliv, 0.100010, 0.140000, 1.000000, 0.140000
 price, 0.100008, 0.180000, 1.000000, 0.180000
 money, 0.100007, 0.180000, 1.000000, 0.180000
 got, 0.100004, 0.020000, 1.000000, 0.020000
 happi, 204.088632, 0.050000, 1.000000, 0.050000
 qualiti, 369.032063, 0.030000, 1.000000, 0.030000
 like, 0.100004, 0.110000, 1.000000, 0.110000
 veri, 0.100005, 0.170000, 1.000000, 0.170000
 amazon, 154.182994, 0.060000, 0.850000, 0.070588
 worth, 0.100002, 0.220000, 1.000000, 0.220000
 bit, 0.100009, 0.140000, 1.000000, 0.140000
 shown, 0.100008, 0.160000, 1.000000, 0.160

 nice, 0.100003, 0.210000, 1.000000, 0.210000
 product, 40.572881, 0.090000, 1.000000, 0.090000
 time, 0.100007, 0.220000, 0.880000, 0.250000
 nice product, 726.123654, 0.160000, 0.750000, 0.213333
 veri, 0.100002, 0.020000, 1.000000, 0.020000
 amazon, 0.100010, 0.150000, 0.850000, 0.176471
 deliv, 0.100003, 0.100000, 1.000000, 0.100000
 price, 0.100002, 0.110000, 1.000000, 0.110000
 excel, 0.100005, 0.080000, 1.000000, 0.080000
 happi, 0.100002, 0.100000, 1.000000, 0.100000
 kid, 0.100006, 0.130000, 0.870000, 0.149425
 buy, 0.100002, 0.040000, 1.000000, 0.040000
 soft, 0.100007, 0.120000, 1.000000, 0.120000
 got, 0.100008, 0.060000, 1.000000, 0.060000
 qualiti, 589.675490, 0.010000, 1.000000, 0.010000
 good, 0.100010, 0.160000, 1.000000, 0.160000
 qualiti product, 0.100012, 0.070000, 0.790000, 0.088608
 colour, 0.100008, 0.050000, 1.000000, 0.050000
 's, 0.100005, 0.130000, 1.000000, 0.130000
 gift, 0.100004, 0.070000, 1.000000, 0.070000
 money, 0.100009, 0.140000, 1.000000, 0.140000


In [47]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, vectorizer)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      17.551897        1       1 -0.188496 -0.097680
7      11.831850        1       2  0.131614 -0.255743
8      10.910887        1       3 -0.194067 -0.087044
5       9.572117        1       4 -0.068558  0.029020
4       9.542852        1       5  0.147010 -0.194379
0       9.374072        1       6 -0.262166  0.124173
2       9.171596        1       7  0.116003 -0.159508
6       8.929459        1       8 -0.183030  0.187095
3       7.037929        1       9  0.239827  0.212471
1       6.077341        1      10  0.261863  0.241595, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
27    Default   822.000000          love   822.000000  30.0000  30.0000
31    Default   714.000000          nice   714.000000  29.0000  29.0000
37    Default  1132.000000       qualiti  1132.000000  28.0000  28.0000
36    Default  1783.000000       product  1783.000000  27.0000  27.0000
16    Default  1959.000000          good  1959.000000  26.0000  26.0000
52    Default  1487.000000          veri  1487.000000  25.0000  25.0000
50    Default   504.000000           toy   504.000000  24.0000  24.0000
9     Default   465.000000          cute   465.000000  23.0000  23.0000
44    Default   430.000000         small   430.000000  22.0000  22.0000
1     Default   401.000000        amazon   401.000000  21.0000  21.0000
24    Default   504.000000          like   504.000000  20.0000  20.0000
23    Default   369.000000           kid   369.000000  19.0000  19.0000
48    Default   318.000000         thank   318.000000  18.0000  18.0000
43    Default   400.000000          size   400.000000  17.0000  17.0000
30    Default   379.000000           n't   379.000000  16.0000  16.0000
0     Default   532.000000            's   532.000000  15.0000  15.0000
34    Default   523.000000        pictur   523.000000  14.0000  14.0000
15    Default   276.000000          gift   276.000000  13.0000  13.0000
45    Default   628.000000          soft   628.000000  12.0000  12.0000
41    Default   425.000000         shown   425.000000  11.0000  11.0000
26    Default   599.000000          look   599.000000  10.0000  10.0000
55    Default   308.000000         worth   308.000000   9.0000   9.0000
47    Default   341.000000         teddi   341.000000   8.0000   8.0000
40    Default   262.000000        receiv   262.000000   7.0000   7.0000
19    Default   295.000000           got   295.000000   6.0000   6.0000
14    Default   290.000000        expect   290.000000   5.0000   5.0000
13    Default   264.000000         excel   264.000000   4.0000   4.0000
22    Default   309.000000          just   309.000000   3.0000   3.0000
11    Default   265.000000        differ   265.000000   2.0000   2.0000
39    Default   310.000000        realli   310.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
28    Topic10     0.109138        materi   218.426600  -4.8010  -9.4554
2     Topic10     0.109144        awesom   224.502683  -4.8284  -9.4554
3     Topic10     0.109152          babi   224.780359  -4.8295  -9.4553
8     Topic10     0.109132        colour   232.770999  -4.8646  -9.4555
4     Topic10     0.109132           bad   234.075527  -4.8702  -9.4555
17    Topic10     0.109136  good product   255.557611  -4.9580  -9.4555
54    Topic10     0.109142     veri good   258.842937  -4.9707  -9.4554
40    Topic10     0.109137        receiv   262.798117  -4.9859  -9.4554
11    Topic10     0.109132        differ   265.194728  -4.9951  -9.4555
14    Topic10     0.109134        expect   290.343865  -5.0856  -9.4555
19    Topic10     0.109140           got   295.539191  -5.1033  -9.4554
36    Topic10     0.109850       product  1783.091278  -6.8941  -9.4489
52    Topic10     0.109152          veri  1487.206342  -6.7191  -9.45

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-03-03 15:00:05.786252. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [48]:
# #挑選特定條件(抽樣)
# samples = ldaClusterframe[ldaClusterframe.loc[:,'ClusterLabel'] == 4][:441]
# Rsamples = samples['review'].values
# Rsamples

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-03-03 15:00:07.196266. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [49]:
# with open('../test.csv','w',encoding='utf-8') as f :
#     f.write(str(Rsamples))

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-03-03 15:00:07.204275. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
